In [5]:
import pathlib
import stat

import rich

# https://wiki.python.org/moin/BitwiseOperators
file_attrs = pathlib.Path(".").stat().st_file_attributes

file_attrs & stat.FILE_ATTRIBUTE_ARCHIVE == stat.FILE_ATTRIBUTE_ARCHIVE

file_attrs & stat.FILE_ATTRIBUTE_ARCHIVE != 0

print(f"0b{file_attrs:b}")

0b10000


0b0000 == 0

AND GATE
--------
0b0000 & <ANY> == 0  AND GATE

0b0000 & 0b1111 == 0b1111  OR GATE

In [12]:
from dataclasses import dataclass
from pathlib import Path

import win32api
from rich import print

@dataclass(order=True, frozen=True, slots=True)
class FileVersion:
    major: int
    minor: int
    patch: int
    label: int

    def __init__(self, path: str):
        # take the `path` and resolve it to the fully-qualified path
        path: Path = Path(path).resolve()

        if not path.exists():
            raise Exception("path doesn't exist")

        if path.suffix != ".exe":
            raise Exception("path must be .exe")

        # https://stackoverflow.com/a/1237635
        # the `\` represents which information to return from the VS_FIXEDFILEINFO struct, the "SubBlock"
        # https://docs.microsoft.com/en-us/windows/win32/api/verrsrc/ns-verrsrc-vs_fixedfileinfo#syntax
        version_info = win32api.GetFileVersionInfo(str(path), "\\")

        # we cannot use simple field assignment due to `frozen=True`
        # https://docs.python.org/3/library/dataclasses.html#frozen-instances
        object.__setattr__(self, "major", win32api.HIWORD(version_info["FileVersionMS"]))
        object.__setattr__(self, "minor", win32api.LOWORD(version_info["FileVersionMS"]))
        object.__setattr__(self, "patch", win32api.HIWORD(version_info["FileVersionLS"]))
        object.__setattr__(self, "label", win32api.LOWORD(version_info["FileVersionLS"]))

py310_version = FileVersion(r".venv\Scripts\python.exe")
py36_version = FileVersion(r"C:\Program Files\Python36\python.exe")

# info = win32api.GetFileVersionInfo(".venv\\Scripts\\python.exe", "\\")
# file_version_ms = info["FileVersionMS"]
# file_version_ls = info["FileVersionLS"]

# major = win32api.HIWORD(file_version_ms)
# minor = win32api.LOWORD(file_version_ms)
# patch = win32api.HIWORD(file_version_ls)
# label = win32api.LOWORD(file_version_ls)

# py310_version = FileVersion(major, minor, patch, label)

# info = win32api.GetFileVersionInfo("C:\\Program Files\\Python36\\python.exe", "\\")
# file_version_ms = info["FileVersionMS"]
# file_version_ls = info["FileVersionLS"]

# major = win32api.HIWORD(file_version_ms)
# minor = win32api.LOWORD(file_version_ms)
# patch = win32api.HIWORD(file_version_ls)
# label = win32api.LOWORD(file_version_ls)

# py36_version = FileVersion(major, minor, patch, label)

print(py36_version > py310_version)
print(py36_version)
print(py310_version)

False

FileVersion(major=3, minor=6, patch=8150, label=1013)

FileVersion(major=3, minor=10, patch=2150, label=1013)